# Advent of code

### Problem 1 ( Minimum Total Distance)
You are given two lists of integers, left and right. To calculate the total distance between the two lists:
Sort both lists in ascending order.
Pair each element in the sorted left list with the corresponding element in the sorted right list.
Calculate the absolute difference for each pair.
Return the sum of all these differences as the result.

In [1]:
with open('input_day1.txt', 'r') as file:
    column1 = []
    column2 = []
    for line in file:
        values = line.split()
        if len(values) == 2:
            column1.append(int(values[0]))
            column2.append(int(values[1]))

column1 = sorted(column1)
column2 = sorted(column2)
distance = [abs(x - y) for x, y in zip(column1, column2)]
sum(distance)

2815556

### Problem 2 (Similarity Score)
You are given two lists of integers, left and right. To calculate the similarity score:
Count how many times each number appears in right.
For each number in left, multiply the number by the count of its occurrences in right.
Sum these products to get the final similarity score.

In [2]:
sum([x * column2.count(x) for x in column1])

23927637

### Problem 3 (Determine Safe Reports)
You are given a list of reports, where each report is a list of integers representing "levels." A report is considered safe if:

* Monotonicity: All levels are either strictly increasing or strictly decreasing.
* Valid Differences: The absolute difference between any two adjacent levels is at least 1 and at most 3.
You need to count how many reports in the dataset are safe.

In [3]:
def increasing(L):
    return all(x<=y for x, y in zip(L, L[1:]))

def decreasing(L):
    return all(x>=y for x, y in zip(L, L[1:]))

def monotonic(L):
    return increasing(L) or decreasing(L)

with open('input_day2.txt', 'r') as file:
    reports = []
    for line in file:
        reports.append( list(map(int, line.split()) ))

In [4]:
def differ(L):
    diff_list = []
    for i in range(len(L)-1):
        diff_list.append(abs(L[i+1] - L[i]))
    return diff_list

In [5]:
safety = 0
for report in reports:
    if monotonic(report):
        diff_list = differ(report)
        if max(diff_list) <4 and min(diff_list) > 0:
            safety += 1
print(safety)

660


### Problem 4

Modify the analysis to account for the Problem Dampener, which allows the safety system to ignore one bad level. If removing one level from an unsafe report makes it safe, the report is considered safe.



In [7]:
#lazy_solution
safety = 0

def is_safe(report):
    if monotonic(report):
        diff_list = differ(report)
        if max(diff_list) <4 and min(diff_list) > 0:
            return True
    return False

for report in reports:
    if is_safe(report):
        safety += 1
    else:
        for i in range(len(report)):
            modified_report = report.copy()
            modified_report.pop(i)
            if is_safe(modified_report):
                safety += 1
                break

In [8]:
safety

689

### Problem 5 (Parse and Calculate Valid Multiplications)

Given a corrupted string of memory, extract valid multiplication instructions in the form mul(X,Y) and calculate the sum of their results. Invalid or malformed instructions should be ignored.

**Rules for Valid Instructions:**

1. The instruction must match the format: mul(X,Y)
* X and Y are integers (1 to 3 digits).
* No extra characters or spaces are allowed within the parentheses.
2. Ignore malformed instructions (e.g., mul[3,7], mul(4*), or mul ( 2 , 4 )).

In [9]:
import re
pattern = r"mul\((\d{1,3}),(\d{1,3})\)"
with open('input_day3.txt', 'r') as file:
    memory = file.read()
matches = re.findall(pattern, memory)
total = sum([int(x) * int(y) for x, y in matches])
total

188741603

### Problem 6

* The do() command enables all future mul instructions.
* The don't() command disables all future mul instructions.
* By default, multiplication operations are enabled.

In [10]:
dos_donts = r"do\(\)|don't\(\)"
dos_donts_matches = [(g.start(), g.group()) for g in re.compile(dos_donts).finditer(memory)]
pattern_matches = [(m.start(), m.group()) for m in re.compile(pattern).finditer(memory)]
#https://stackoverflow.com/questions/6100817/how-to-make-python-regex-which-matches-multiple-patterns-to-same-index

In [12]:
# we want to make these into an indicator function
# Convert "do()" to 1 and "don't()" to 0
indicator_list = [(index, 1 if value == 'do()' else 0) for index, value in dos_donts_matches]

# Remove consecutive duplicates
filtered_list = [indicator_list[0]]
for item in indicator_list[1:]:
    if item[1] != filtered_list[-1][1]:
        filtered_list.append(item)
filtered_list.insert(0, (0, 1))
filtered_list[:3]


[(0, 1), (402, 0), (572, 1)]

In [13]:
def indicator_function(index, filtered_list):
    for i in range(len(filtered_list) - 1):
        if filtered_list[i][0] <= index < filtered_list[i + 1][0]:
            return filtered_list[i][1]
    return filtered_list[-1][1]

filtered_values = [indicator_function(index, filtered_list)*int(re.search(pattern, value).group(1))*int(re.search(pattern, value).group(2))  for index, value in pattern_matches]
filtered_values[:3]

[265734, 242844, 11225]

In [14]:
sum(filtered_values)

67269798

In [15]:

# Ciaran's solution
regex = re.compile(r"mul\((\d+),(\d+)\)|do\(\)|don't\(\)")
enabled = True
total = 0

for match in regex.finditer(memory):
    result = match.groups()
    if match.group(0) == "do()":
        enabled = True
    elif match.group(0) == "don't()":
        enabled = False
    else:
        if enabled:
            total += int(result[0]) * int(result[1])

total


67269798